In [2]:
import numpy as np # linear algebra
import fasttext # language detection
from huggingface_hub import hf_hub_download # download models from huggingface hub
from nltk.tokenize import word_tokenize # tokenize words

### API:
- Filtering functions should return `True` or `False`
    - Some will need threshold tuning, so some of them will return two functions, one of which will return number
- Cleaning functions should take document as input and return cleaned document.

### Cleaning

In [6]:
def standardize_whitespace(document: str):
    whitespaces = {
        " ",
        " ",
        " ",
        " ",
        " ",
        "　",
        " ",
        " ",
        " ",
        " ",
        "￼",
        "",
    }
    document = "".join([char if char not in whitespaces else " " for char in document])
    return document


standardize_whitespace("salam necəsən? Mən Azərbaycan dilində yazılmış bir proqramam.")

'salam necəsən? Mən Azərbaycan dilində yazılmış bir proqramam.'

### Filtering

In [6]:
### language filtering with `fasttext`
# https://huggingface.co/facebook/fasttext-language-identification
lang_detect_model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(lang_detect_model_path)

def detect_language(text):
    """Higher the better"""
    pred = model.predict(text)
    language = pred[0][0].replace("__label__", "")
    is_azerbaijani = True if language == "azj_Latn" else False
    return is_azerbaijani, pred[1][0]

# detect_language("salam necəsən? Mən Azərbaycan dilində yazılmış bir proqramam.")
detect_language(input("Enter a text: "))

(True, 1.0000059604644775)

In [33]:
# Filtering documents on the number of words. Use `nltk.word_tokenize`.
def get_word_count(document: str):
    return len(word_tokenize(document))

def filter_document_on_word_count(word_count, min_word_count, max_word_count):
    if min_word_count <= word_count <= max_word_count:
        return True
    return False

document = "salam necəsən? Mən Azərbaycan dilində yazılmış bir proqramam.   askmaksmn\ndfeasfs"
word_tokenize(document)

['salam',
 'necəsən',
 '?',
 'Mən',
 'Azərbaycan',
 'dilində',
 'yazılmış',
 'bir',
 'proqramam',
 '.',
 'askmaksmn',
 'dfeasfs']

In [20]:
from collections import Counter
from math import sqrt

def calculate_character_repetition_ratio(document, n=10, j=2):
    n_grams = [document[i:i+n] for i in range(len(document) - n + 1)] # generate n-grams
    n_gram_frequencies = Counter(n_grams) # count frequencies of n-grams
    nn_grams = len(n_gram_frequencies) # calculate nn-grams
    nrep_n_grams = int(sqrt(nn_grams)) if j == 2 else int(nn_grams**(1/j)) # calculate nrep-n-grams
    # calculate character repetition ratio
    most_common_nrep = n_gram_frequencies.most_common(nrep_n_grams)
    sum_most_common_nrep = sum(freq for _, freq in most_common_nrep)
    total_sum_n_grams = sum(n_gram_frequencies.values())
    character_repetition_ratio = sum_most_common_nrep / total_sum_n_grams
    return character_repetition_ratio

# Example usage
document = "This is an example document to test the character repetition ratio."
document = "My name is Eljan and this document is written in English. And my name is Eljan hahah. And his name is Eljan hahah."
calculate_character_repetition_ratio(document)


0.22857142857142856

In [31]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("eljanmahammadli/azGPT-perplexity")

In [32]:
tokenizer.tokenize("salam necəsən? Mən Azərbaycan dilində yazılmış bir proqramam.")

['▁sal',
 'am',
 '▁n',
 'ec',
 'əsən',
 '?',
 '▁Mən▁',
 'Azərbaycan▁dilində▁',
 'yazılmış',
 '▁bir▁',
 'proqram',
 'am',
 '.']

In [3]:
from datasets import load_from_disk

ds = load_from_disk("/Users/eljan/Documents/azGPT/data/temp/az")

ds

Dataset({
    features: ['id', 'text', 'title'],
    num_rows: 1213
})

In [6]:
ds.to_pandas()

,id,text,title
0,696123,Maşa - obyektləri qavrayıb yerini dəyişdirmək ...,Maşa
1,255121,Kəlbəli xan Əhməd xan oğlu Xoyski Rus ordusund...,Kəlbəli xan Xoyski
2,506863,Depersonalizasiya psixoloji xəstəlik növü. Bu ...,Depersonalizasiya
3,813924,Seek and Destroy Takara Co Ltd. tərəfindən bur...,Seek and Destroy (2002 video oyun)
4,279201,Merilin Cess Fransa pornoaktrisası. 70-dən çox...,Merilin Cess
...,...,...,...
1208,181631,Üç yüz altı say sistemində ədədlərdən biridir....,306 (ədəd)
1209,358827,Vanna Buakeva İqtisadçının şərəfini qoruyan tə...,Vanna Buakeva
1210,436735,"Fredrik Bayer Danimarka yazıçısı, müəllim və p...",Fredrik Bayer
1211,709421,"Danimarka Vest-Hind şirkəti, və ya Danimarka V...",Vest-Hind şirkəti (Danimarka)
